In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append("/home/jinsu/workstation/project/debiasing-multi-modal")


df_zs_embeddings = pd.read_json("/home/jinsu/workstation/project/debiasing-multi-modal/data/embeddings/waterbirds/RN50/embedding_prediction.json")
df_meta = pd.read_csv("/home/jinsu/workstation/project/debiasing-multi-modal/data/waterbirds/waterbird_complete95_forest2water2/metadata.csv")

In [2]:
df_zs_embeddings

,001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0074_59.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0014_89.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0031_100.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0010_796097.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0023_796059.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0040_796066.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0089_796069.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0067_170.jpg,001.Black_footed_Albatross/Black_Footed_Albatross_0060_796076.jpg,...,200.Common_Yellowthroat/Common_Yellowthroat_0021_190655.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0011_190401.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0029_190403.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0071_190665.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0070_190678.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0040_190427.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0063_190440.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0058_190958.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0008_190703.jpg,200.Common_Yellowthroat/Common_Yellowthroat_0055_190967.jpg
y,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
place,1,1,1,1,1,1,1,1,1,1,...,1,0,0,1,0,1,1,0,0,1
group,3,3,3,3,3,3,3,3,3,3,...,1,0,0,1,0,1,1,0,0,1
split,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
image_embedding,"[0.012435913085937, 0.007781982421875001, 0.02...","[-0.012657165527343, -0.015228271484375002, 0....","[-0.007061004638671001, -0.007820129394531, -0...","[-0.000501632690429, -0.009048461914062, 0.025...","[-0.002298355102539, 0.008895874023437, 0.0137...","[-0.017959594726562, -0.0135498046875, 0.00348...","[-0.020599365234375, 0.003993988037109, -0.014...","[-0.020538330078125, -0.026107788085937, 0.023...","[-0.0031890869140620004, -0.013809204101562, 0...","[-0.01031494140625, 3.343820571899414e-05, 0.0...",...,"[0.01251220703125, 0.024337768554687, -0.00127...","[0.0169677734375, 0.020248413085937, -0.005756...","[-0.007366180419921001, 0.034454345703125, 0.0...","[-0.0035343170166010004, 0.004840850830078, -0...","[0.012901306152343, 0.024017333984375003, 0.01...","[-0.000723361968994, -0.010108947753906, -0.02...","[0.005821228027343001, 0.00714111328125, -0.00...","[-0.007404327392578001, 0.020523071289062, 0.0...","[0.014686584472656002, 0.001265525817871, 0.01...","[0.012840270996093, 0.01300048828125, 0.013381..."
y_pred,0,1,1,1,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
import torch
import torch.nn as nn
from __future__ import print_function

import sys
import argparse
import time
import tqdm
import math

import torch
import torch.backends.cudnn as cudnn

from util import AverageMeter
from util import adjust_learning_rate, warmup_learning_rate, accuracy, accuracy_zs
from util import set_optimizer
# from networks.resnet_big import SupConResNet, LinearClassifier

try:
    import apex
    from apex import amp, optimizers
except ImportError:
    pass

from resnet import resnet50
from data.waterbirds_embeddings import Embeddings, load_embeddings
model_dict = {'resnet50': [resnet50, 1024]}
new_order_for_print = [
    'weighted_mean_acc',
    'worst_acc',
    'acc_0_0',
    'acc_0_1',
    'acc_1_0',
    'acc_1_1',
    'mean_acc'
]
from functools import partial

class LinearClassifier(nn.Module):
    def __init__(self, name='resnet50', num_classes=2):
        super(LinearClassifier, self).__init__()
        _, feat_dim = model_dict[name]
        self.fc = nn.Linear(feat_dim, num_classes)

    def forward(self, features):
        return self.fc(features)

def parse_option():
    parser = argparse.ArgumentParser('argument for training')

    parser.add_argument('--print_freq', type=int, default=10,
                        help='print frequency')
    parser.add_argument('--save_freq', type=int, default=50,
                        help='save frequency')
    parser.add_argument('--batch_size', type=int, default=256,
                        help='batch_size')
    parser.add_argument('--num_workers', type=int, default=16,
                        help='num of workers to use')
    parser.add_argument('--epochs', type=int, default=5,
                        help='number of training epochs')

    # optimization
    parser.add_argument('--learning_rate', type=float, default=0.1,
                        help='learning rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='60,75,90',
                        help='where to decay lr, can be a list')
    parser.add_argument('--lr_decay_rate', type=float, default=0.2,
                        help='decay rate for learning rate')
    parser.add_argument('--weight_decay', type=float, default=0,
                        help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9,
                        help='momentum')

    # model dataset
    parser.add_argument('--model', type=str, default='resnet50')
    parser.add_argument('--dataset', type=str, default='waterbirds',
                        choices=['celebA', 'waterbirds'], help='dataset')

    # other setting
    parser.add_argument('--cosine', action='store_true',
                        help='using cosine annealing')
    parser.add_argument('--warm', action='store_true',
                        help='warm-up for large batch training')

    parser.add_argument('--embedding_dir', type=str,
                        help='extracted embedding')
    parser.add_argument('--target', type=str, default="class", choices=["class", "group", "spurious"])
    parser.add_argument('--data_dir', type=str,
                        help='metadata.csv')

    opt = parser.parse_args(args=[])

    # set the path according to the environment

    iterations = opt.lr_decay_epochs.split(',')
    opt.lr_decay_epochs = list([])
    for it in iterations:
        opt.lr_decay_epochs.append(int(it))

    opt.model_name = '{}_{}_lr_{}_decay_{}_bsz_{}'.\
        format(opt.dataset, opt.model, opt.learning_rate, opt.weight_decay,
               opt.batch_size)

    if opt.cosine:
        opt.model_name = '{}_cosine'.format(opt.model_name)

    # warm-up for large-batch training,
    if opt.warm:
        opt.model_name = '{}_warm'.format(opt.model_name)
        opt.warmup_from = 0.01
        opt.warm_epochs = 10
        if opt.cosine:
            eta_min = opt.learning_rate * (opt.lr_decay_rate ** 3)
            opt.warmup_to = eta_min + (opt.learning_rate - eta_min) * (
                    1 + math.cos(math.pi * opt.warm_epochs / opt.epochs)) / 2
        else:
            opt.warmup_to = opt.learning_rate
            
    if opt.dataset == 'celebA':
        opt.n_cls = 2
    elif opt.dataset == 'waterbirds':
        opt.n_cls = 2
    else:
        raise ValueError('dataset not supported: {}'.format(opt.dataset))

    return opt


def set_model(opt):
    # model = SupConResNet(name=opt.model)
    criterion = torch.nn.CrossEntropyLoss()

    classifier = LinearClassifier(name=opt.model, num_classes=opt.n_cls)

    # ckpt = torch.load(opt.ckpt, map_location='cpu')
    # state_dict = ckpt['model']

    if torch.cuda.is_available():
        # if torch.cuda.device_count() > 1:
        #     model.encoder = torch.nn.DataParallel(model.encoder)
        # else:
            # new_state_dict = {}
            # for k, v in state_dict.items():
            #     k = k.replace("module.", "")
            #     new_state_dict[k] = v
            # state_dict = new_state_dict
        
        # model = model.cuda()
        classifier = classifier.cuda()
        criterion = criterion.cuda()
        cudnn.benchmark = True

        # model.load_state_dict(state_dict)

    return classifier, criterion # model, 

def update_dict(acc_groups, y, g, logits):
    preds = torch.argmax(logits, axis=1)
    correct_batch = (preds == y)
    g = g.cpu()
    for g_val in np.unique(g):
        mask = g == g_val
        n = mask.sum().item()
        corr = correct_batch[mask].sum().item()
        acc_groups[g_val].update(corr / n, n) # AverageMeter Updater. 

def get_results(acc_groups, get_yp_func, ): # Input 중 acc_groups : AverageMeter()를 담고있는 dict. get_yp_func : 미리 partial을 이용해 n_groups를 저장해놓음. 
    groups = acc_groups.keys() # 0, 1, 2, 3
    results = {
            f"acc_{get_yp_func(g)[0]}_{get_yp_func(g)[1]}": acc_groups[g].avg
            for g in groups
    }
    all_correct = sum([acc_groups[g].sum for g in groups])
    all_total = sum([acc_groups[g].count for g in groups])
    results.update({"mean_acc" : all_correct / all_total})
    results.update({"worst_acc" : min(results.values())})
    
    return results

def get_y_p(g, n_places):
    y = g // n_places
    p = g % n_places
    return y, p


def train(train_loader, classifier, criterion, optimizer, epoch, get_yp_func, target, label='Train'): # model,
    """one epoch training"""
    # model.eval()
    classifier.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()
    acc_groups = {g_idx : AverageMeter() for g_idx in range(train_loader.dataset.n_groups)}

    end = time.time()
    for idx, data in enumerate(train_loader):
        if opt.dataset == 'waterbirds':
            embeddings, all_labels, _ = data
            labels = all_labels[target] # (y, y_group, y_spurious)
            groups = all_labels['group']
        else:
            embeddings, all_labels = data
            labels = all_labels[target] # (y, y_group, y_spurious)
            groups = all_labels['group']
        
        data_time.update(time.time() - end)

        embeddings = embeddings.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)
        bsz = labels.shape[0]

        # warm-up learning rate
        warmup_learning_rate(opt, epoch, idx, len(train_loader), optimizer)

        # compute loss
        # with torch.no_grad():
        #     features = model.encoder(embeddings)
        output = classifier(embeddings.detach())
        loss = criterion(output, labels)

        # update metric
        losses.update(loss.item(), bsz)
        acc1 = accuracy(output, labels, bsz)
        acc.update(acc1, bsz)

        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        # Update acc dict
        update_dict(acc_groups, labels, groups, output)
        
        # print info
        if (idx + 1) % opt.print_freq == 0:
            print(f'{label}: [{0}][{1}/{2}]\t'
                  'BT {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'DT {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'loss {loss.val:.3f} ({loss.avg:.3f})\t'
                  'Acc@1 {acc.val:.3f} ({acc.avg:.3f})'.format(
                   epoch, idx + 1, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, acc=acc))
            sys.stdout.flush()
            
    group_acc = get_results(acc_groups, get_yp_func) # NOTE declared in [def main]
    group_acc = {key: group_acc[key] for key in new_order_for_print[1:]}
    print(f"{label}:", str(group_acc))
    
    return losses.avg, acc.avg, group_acc


def validate(val_loader, classifier, criterion, get_yp_func, train_group_ratio, target, label='Test', watch=True):
    """validation"""
    
    # model.eval()
    classifier.eval()

    batch_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()
    acc_groups = {g_idx : AverageMeter() for g_idx in range(val_loader.dataset.n_groups)}

    with torch.no_grad():
        end = time.time()
        for idx, data in enumerate(val_loader):
            if opt.dataset == 'waterbirds':
                embeddings, all_labels, _ = data
                labels = all_labels[target] # (y, y_group, y_spurious)
                groups = all_labels['group']
            else:
                embeddings, all_labels = data
                labels = all_labels[target] # (y, y_group, y_spurious)
                groups = all_labels['group']
            
            embeddings = embeddings.float().cuda()
            labels = labels.cuda()
            bsz = labels.shape[0]

            # forward
            output = classifier(embeddings)
            loss = criterion(output, labels)

            # update metric
            losses.update(loss.item(), bsz)
            acc1 = accuracy(output, labels, bsz)
            acc.update(acc1, bsz)

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            
            # Update acc dict
            update_dict(acc_groups, labels, groups, output)
        
            if (idx+1) % opt.print_freq == 0:
                if watch:
                    print(f'{label}: [{0}/{1}]\t'
                        'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                        'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                        'Acc@1 {acc.val:.3f} ({acc.avg:.3f})'.format(
                        idx, len(val_loader), batch_time=batch_time,
                        loss=losses, acc=acc))
                    
                    
    group_acc = get_results(acc_groups, get_yp_func)
    
    #NOTE add Weighted mean acc.
    groups = range(val_loader.dataset.n_groups) # 0, 1, 2, 3
    group_acc_indiv =  [group_acc[f"acc_{get_yp_func(g)[0]}_{get_yp_func(g)[1]}"] for g in groups]
    weighted_mean_acc = (np.array(group_acc_indiv) * np.array(train_group_ratio)).sum() # Weighted Sum \
    
    group_acc["weighted_mean_acc"] = weighted_mean_acc
    group_acc = {key: group_acc[key] for key in new_order_for_print}
    
    if watch:
        print(f"{label}:", str(group_acc))
        print(' * Acc@1 {acc.avg:.3f}'.format(acc=acc))
    return losses.avg, acc.avg, group_acc


def main(opt):
    best_acc = 0
    best_epoch = 0
    # opt = parse_option()

    # build dataset example.
    trainset = Embeddings(opt.data_dir, 'train', opt.embedding_dir, None, None)
    get_yp_func = partial(get_y_p, n_places=trainset.n_places)
    train_group_ratio = trainset.group_ratio
    
    # build data loader
    print("Load Data Loader (train, validation, test)")
    train_loader, val_loader, test_loader = load_embeddings(opt.data_dir, opt.embedding_dir, opt.batch_size, opt.batch_size)
    
    # build model and criterion
    print("Set Linear Classifier")
    classifier, criterion = set_model(opt) # model, 

    # build optimizer
    print("Set Optimizer")
    optimizer = set_optimizer(opt, classifier)

    # training routine
    train_losses = []
    train_accs = []
    train_group_accs = []
    val_losses = []
    val_accs = []
    val_group_accs = []
    
    test_losses_y = [] # NOTE: Don't peek ! 
    test_accs_y = [] # NOTE: Don't peek ! 
    test_group_accs_y = [] # NOTE: Don't peek ! 
    test_losses_spurious = [] # NOTE: Don't peek ! 
    test_accs_spurious = [] # NOTE: Don't peek ! 
    test_group_accs_spurious = [] # NOTE: Don't peek ! 
    
    for epoch in range(1, opt.epochs + 1):
        adjust_learning_rate(opt, optimizer, epoch)


        print(f'--- Epoch {epoch} ---')
        # train for one epoch
        loss, acc, group_acc = train(train_loader, classifier, criterion,
                          optimizer, epoch, get_yp_func, target='y', label='Train(y)')
        
        train_losses.append(loss)
        train_accs.append(acc)
        train_group_accs.append(group_acc)
        # eval for one epoch
        val_loss, val_acc, val_group_acc = validate(val_loader, classifier, criterion, get_yp_func, train_group_ratio, target='y', label='Val(y)')
        if val_group_acc['worst_acc'] > best_acc:
            best_acc = val_group_acc['worst_acc']
            best_epoch = epoch
        
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        val_group_accs.append(val_group_acc)
            
        test_loss_y, test_acc_y, test_group_acc_y = validate(test_loader, classifier, criterion, get_yp_func, train_group_ratio, target='y', label='Test(y)', watch=True)
        test_losses_y.append(test_loss_y)
        test_accs_y.append(test_acc_y)
        test_group_accs_y.append(test_group_acc_y)
        
        test_loss_spurious, test_acc_spurious, test_group_acc_spurious= validate(test_loader, classifier, criterion, get_yp_func, train_group_ratio, target='spurious', label='Test(spurious)', watch=True)
        test_losses_spurious.append(test_loss_spurious)
        test_accs_spurious.append(test_acc_spurious)
        test_group_accs_spurious.append(test_group_acc_spurious)
    
    
    print('==================================================================')
    print('best epoch : {}'.format(best_epoch))
    print('best (worst-)validation accuracy: {} '.format(val_group_accs[best_epoch-1]))
    
    print('best test accuracy (class): {}'.format(test_group_accs_y[best_epoch-1]))
    print('best test accuracy (spurious): {}'.format(test_group_accs_spurious[best_epoch-1]))

/home/jinsu/anaconda3/envs/cuda_test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import argparse

parser = argparse.ArgumentParser('argument for training')

parser.add_argument('--print_freq', type=int, default=20,
                    help='print frequency')
parser.add_argument('--save_freq', type=int, default=50,
                    help='save frequency')
parser.add_argument('--batch_size', type=int, default=512,
                    help='batch_size')
parser.add_argument('--num_workers', type=int, default=16,
                    help='num of workers to use')
parser.add_argument('--epochs', type=int, default=10,
                    help='number of training epochs')

# optimization
parser.add_argument('--learning_rate', type=float, default=5,
                    help='learning rate')
parser.add_argument('--lr_decay_epochs', type=str, default='60,75,90',
                    help='where to decay lr, can be a list')
parser.add_argument('--lr_decay_rate', type=float, default=0.2,
                    help='decay rate for learning rate')
parser.add_argument('--weight_decay', type=float, default=0,
                    help='weight decay')
parser.add_argument('--momentum', type=float, default=0.9,
                    help='momentum')

# model dataset
parser.add_argument('--model', type=str, default='resnet50')
parser.add_argument('--dataset', type=str, default='waterbirds',
                    choices=['celebA', 'waterbirds'], help='dataset')

# other setting
parser.add_argument('--cosine', action='store_true',
                    help='using cosine annealing')
parser.add_argument('--warm', action='store_true',
                    help='warm-up for large batch training')

parser.add_argument('--embedding_dir', type=str, 
                    help='extracted embedding')
parser.add_argument('--target', type=str, default="class", choices=["class", "group", "spurious"]) # Label for linear proving
parser.add_argument('--data_dir', type=str,
                    help='folder, in which [metadata.csv] exists')

opt = parser.parse_args(args=[])   

iterations = opt.lr_decay_epochs.split(',')
opt.lr_decay_epochs = list([])
for it in iterations:
    opt.lr_decay_epochs.append(int(it))

opt.model_name = '{}_{}_lr_{}_decay_{}_bsz_{}'.\
    format(opt.dataset, opt.model, opt.learning_rate, opt.weight_decay,
            opt.batch_size)

if opt.cosine:
    opt.model_name = '{}_cosine'.format(opt.model_name)

# warm-up for large-batch training,
if opt.warm:
    opt.model_name = '{}_warm'.format(opt.model_name)
    opt.warmup_from = 0.01
    opt.warm_epochs = 10
    if opt.cosine:
        eta_min = opt.learning_rate * (opt.lr_decay_rate ** 3)
        opt.warmup_to = eta_min + (opt.learning_rate - eta_min) * (
                1 + math.cos(math.pi * opt.warm_epochs / opt.epochs)) / 2
    else:
        opt.warmup_to = opt.learning_rate
        
if opt.dataset == 'celebA':
    opt.n_cls = 2
elif opt.dataset == 'waterbirds':
    opt.n_cls = 2
else:
    raise ValueError('dataset not supported: {}'.format(opt.dataset))

# Linear Proving  
- (Epoch default : 100)
- Test 성능 모니터링하면 안 됨.

In [56]:
opt.embedding_dir = "/home/jinsu/workstation/project/debiasing-multi-modal/data/embeddings/waterbirds/RN50/embedding_prediction.json"
opt.data_dir = "/home/jinsu/workstation/project/debiasing-multi-modal/data/waterbirds/waterbird_complete95_forest2water2"
if __name__ == '__main__':    
    main(opt)
    

Load Data Loader (train, validation, test)
Set Linear Classifier
Set Optimizer
--- Epoch 1 ---
Train(y): {'worst_acc': 0.16071428571428573, 'acc_0_0': 0.8850771869639794, 'acc_0_1': 0.8369565217391305, 'acc_1_0': 0.16071428571428573, 'acc_1_1': 0.30179754020813626, 'mean_acc': 0.7461939520333681}
Val(y): {'weighted_mean_acc': 0.7685754887177203, 'worst_acc': 0.05150214592274678, 'acc_0_0': 0.7366167023554604, 'acc_0_1': 0.05150214592274678, 'acc_1_0': 0.8947368421052632, 'acc_1_1': 0.9924812030075187, 'mean_acc': 0.5162635529608006}
 * Acc@1 0.516
Test(y): {'weighted_mean_acc': 0.7550782425464763, 'worst_acc': 0.04567627494456763, 'acc_0_0': 0.7170731707317073, 'acc_0_1': 0.04567627494456763, 'acc_1_0': 0.9252336448598131, 'acc_1_1': 0.9953271028037384, 'mean_acc': 0.5096651708664135}
 * Acc@1 0.510
Test(spurious): {'weighted_mean_acc': 0.7800135852513611, 'worst_acc': 0.07476635514018691, 'acc_0_0': 0.7170731707317073, 'acc_0_1': 0.9543237250554324, 'acc_1_0': 0.07476635514018691, 'ac

# Zero-Shot Evaluation

In [19]:
def update_dict_zs(acc_groups, y, g, preds):
    # preds = torch.argmax(logits, axis=1)
    correct_batch = (preds == y)
    g = g.cpu()
    for g_val in np.unique(g):
        mask = g == g_val
        n = mask.sum().item()
        corr = correct_batch[mask].sum().item()
        acc_groups[g_val].update(corr / n, n) # AverageMeter Updater. 

def validate_zs(val_loader, get_yp_func, train_group_ratio, target, label='Test', watch=True):
    """validation"""

    batch_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()
    acc_groups = {g_idx : AverageMeter() for g_idx in range(val_loader.dataset.n_groups)}

    with torch.no_grad():
        end = time.time()
        for idx, data in enumerate(val_loader):
            if opt.dataset == 'waterbirds':
                _, all_labels, _ = data
                labels = all_labels[target] # (y, y_group, y_spurious)
                groups = all_labels['group']
                preds = all_labels['ebd_y_pred']
            else:
                _,  all_labels, _ = data
                labels = all_labels[target] # (y, y_group, y_spurious)
                groups = all_labels['group']
                preds = all_labels['ebd_y_pred']
              
              
            preds = preds.float().cuda()  
            # embeddings = embeddings.float().cuda()
            labels = labels.cuda()
            bsz = labels.shape[0]

            # forward
            # output = classifier(embeddings)
            # loss = criterion(output, labels)

            # update metric
            # losses.update(loss.item(), bsz)
            acc1 = accuracy_zs(preds, labels, bsz)
            acc.update(acc1, bsz)
            
            # Update acc dict
            update_dict_zs(acc_groups, labels, groups, preds)
                    
                    
    group_acc = get_results(acc_groups, get_yp_func)
    
    #NOTE add Weighted mean acc.
    groups = range(val_loader.dataset.n_groups) # 0, 1, 2, 3
    group_acc_indiv =  [group_acc[f"acc_{get_yp_func(g)[0]}_{get_yp_func(g)[1]}"] for g in groups]
    weighted_mean_acc = (np.array(group_acc_indiv) * np.array(train_group_ratio)).sum() # Weighted Sum \
    
    group_acc["weighted_mean_acc"] = weighted_mean_acc
    group_acc = {key: group_acc[key] for key in new_order_for_print}
    
    if watch:
        print(f"{label}:", str(group_acc))
        print(' * Acc@1 {acc.avg:.3f}'.format(acc=acc))
        
    return  acc.avg, group_acc


def main_zs(opt):
    best_acc = 0
    best_epoch = 0
    # opt = parse_option()

    # build dataset example.
    trainset = Embeddings(opt.data_dir, 'train', opt.embedding_dir, None, None)
    get_yp_func = partial(get_y_p, n_places=trainset.n_places)
    train_group_ratio = trainset.group_ratio
    
    # build data loader
    print("Load Data Loader (train, validation, test)")
    train_loader, val_loader, test_loader = load_embeddings(opt.data_dir, opt.embedding_dir, opt.batch_size, opt.batch_size)

    # train for one epoch

    # eval for one epoch
    val_acc, val_group_acc = validate_zs(val_loader, get_yp_func, train_group_ratio, target='y', label='Val(y)')
        
    test_acc_y, test_group_acc_y = validate_zs(test_loader, get_yp_func, train_group_ratio, target='y', label='Test(y)', watch=True)

    
    print('===============================Final Results===============================')
    print('Zero-shot (worst-)validation accuracy: {} '.format(val_group_acc))
    
    print('Zero-shot test accuracy (class): {}'.format(test_group_acc_y))
    # print('best test accuracy (spurious): {}'.format(test_group_accs_spurious[best_epoch-1]))

In [20]:
opt.embedding_dir = "/home/jinsu/workstation/project/debiasing-multi-modal/data/embeddings/waterbirds/RN50/embedding_prediction.json"
opt.data_dir = "/home/jinsu/workstation/project/debiasing-multi-modal/data/waterbirds/waterbird_complete95_forest2water2"
if __name__ == '__main__':    
    main_zs(opt)
    

Load Data Loader (train, validation, test)
Val(y): {'weighted_mean_acc': 0.9450627926530569, 'worst_acc': 0.3233082706766917, 'acc_0_0': 0.9914346895074947, 'acc_0_1': 0.7145922746781116, 'acc_1_0': 0.3233082706766917, 'acc_1_1': 0.8646616541353384, 'mean_acc': 0.7956630525437864}
 * Acc@1 0.796
Test(y): {'weighted_mean_acc': 0.9289761348579215, 'worst_acc': 0.3909657320872274, 'acc_0_0': 0.9804878048780488, 'acc_0_1': 0.7254988913525499, 'acc_1_0': 0.3909657320872274, 'acc_1_1': 0.822429906542056, 'mean_acc': 0.7984121505005177}
 * Acc@1 0.798
===============================Final Results===============================
Zero-shot (worst-)validation accuracy: {'weighted_mean_acc': 0.9450627926530569, 'worst_acc': 0.3233082706766917, 'acc_0_0': 0.9914346895074947, 'acc_0_1': 0.7145922746781116, 'acc_1_0': 0.3233082706766917, 'acc_1_1': 0.8646616541353384, 'mean_acc': 0.7956630525437864} 
Zero-shot test accuracy (class): {'weighted_mean_acc': 0.9289761348579215, 'worst_acc': 0.390965732087